# Historical Predictions of AusEFlux <img align="right" src="https://github.com/cbur24/AusEFlux/blob/master/results/banner_picture.png?raw=True" width="40%">

This notebook contains the workflow for creating historical carbon and water fluxes for Australia through the full length of the MODIS archive (i.e., 2003-2022). It contains six main steps, instructions are provided in the subsections below. Pay close attention to the `Analysis Parameters` sections and ensure paths etc. are correct.

***
**Ideal compute environment:**

Assuming 5-km resolution

- NCI's 'normal' queue
- X-large (24 cores, 95GiB)
- Python 3.10.0
- Python venv: `/g/data/os22/chad_tmp/AusEFlux/env/py310`
- Folders: `gdata/os22+gdata/ub8+gdata/xc0+gdata/gh70`
***
<!-- > **Expected completion time to run all steps: ~3 hours** -->

## Import libraries and set up Dask

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import warnings
warnings.simplefilter(action='ignore')

import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import start_local_dask

In [ ]:
client = start_local_dask(mem_safety_margin='2Gb')
client

## Set up project directory structure

This workflow assumes a specific file/folder structure, here we create that folder structure to support the rest of the process.

Below, enter the `root directory location` where project results and data are stored. If the folders alredy exist then no directories will be created.

In [ ]:
base='/g/data/os22/chad_tmp/AusEFlux/'

In [ ]:
from _utils import create_project_directories
create_project_directories(root_dir=base)

## Step 1: Spatiotemporal harmonisation of input datasets

Most datasets are originally from here: https://dapds00.nci.org.au/thredds/catalog/ub8/au/catalog.html

Dataset from this process are output as annual layers in `data/interim`

<!-- **Expected completion time ~2hrs** -->

### Analysis Parameters

* `base`: Path to where most of the data is stored
* `results`: Path to store interim datasets after they have undergone harmonisation.
* `year_start`: The first year in the series to predict. If predicting for a single year, make _year_start_ and _year_end_ the same.
* `year_end`: The last year in the series to predict. If predicting for a single year, make _year_start_ and _year_end_ the same.

In [ ]:
base = '/g/data/ub8/au/'
results='/g/data/os22/chad_tmp/AusEFlux/data/interim/'
year_start = 2003
year_end = 2022

### Run step 1



In [ ]:
from _harmonisation import spatiotemporal_harmonisation

In [ ]:
spatiotemporal_harmonisation(
    year_start=year_start,
    year_end=year_end,
    base_path=base,
    results_path=results,
    verbose=True
)

## Step 2: Create feature datasets

Combine results of the spatiotemporal harmonisation into temporally stacked netcdf files, and create new features/variables based on the climate (e.g. anomalies) and remote sensing (e.g veg fractions) datasets. 

**Expected completion time ~6 mins**

### Analysis Parameters

* `base`: Path to where the harmonised datasets output from Step 1 are stored. 
* `results`: Path to store temporally stacked netcdf files i.e. where the outputs of Step 2 will be stored
* `exclude`: Variables to exclude from combining. i.e. Some of the variables in `/interim` output in Step 1 are not needed hereafter.

In [ ]:
base = '/g/data/os22/chad_tmp/AusEFlux/data/interim/'
results='/g/data/os22/chad_tmp/AusEFlux/data/5km/'
exclude = ['.ipynb_checkpoints', 'kTavg', 'Tmax', 'Tmin', 'EVI']

### Run step 2

In [ ]:
from _feature_datasets import create_feature_datasets

In [ ]:
create_feature_datasets(
    base=base,
    results_path=results,
    exclude=exclude,
    verbose=True
)

## Step 3: Extract training data

Scrape the TERN server to extract all of the ozflux eddy covariance data, then append remote sensing data by using the coordinates of the flux tower to extract pixel values. 

**Expected completion time ~12 mins**

### Analysis Parameters

* `version`: Version of OzFlux datasets to use, always has the form 'YYYY_v[number]'
* `level`: What level of OzFlux data to use, level 6 is the highest level and has been pre-processed to 'analysis ready'
* `type` : Ozflux data comes as either 'default' or 'site_pi' depending on how it was processed.
* `rs_data_folder`: Where are the spatiotemporally harmonised and stacked feature layers that we will append to the EC data? The code simple loops through all netcdf files and appends the data. We can filter for features later on.
* `save_ec_data`: If this variables is not 'None', then the EC netcdf files will be exported to this folder.
* `export_path`: Where should we save the .csv files that contain the EC and RS data? i.e. this is our training data
  

In [ ]:
version='2023_v1'
level='L6'
type='default'
rs_data_folder='/g/data/os22/chad_tmp/AusEFlux/data/5km/'
save_ec_data='/g/data/os22/chad_tmp/AusEFlux/data/ozflux_netcdf/'
export_path='/g/data/os22/chad_tmp/AusEFlux/data/training_data/'

### Run Step 3

In [ ]:
from _training import extract_ozflux

In [ ]:
extract_ozflux(
    version=version,
    level=level,
    type=type,
    rs_data_folder=rs_data_folder,
    save_ec_data=save_ec_data,
    export_path=export_path,
    return_coords=True,
    verbose=True
)

### Create a plot of all the OzFlux sites (Optional)

This is helpful in ensuring the site locations are in the correct places - sometimes OzFlux coordinates are incorrect.

We also export a .csv with the site locations

In [ ]:
site_export = '/g/data/os22/chad_tmp/AusEFlux/data/'

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import contextily as ctx

In [ ]:
sites = os.listdir(export_path)

td = []
for site in sites:
    if '.csv' in site:
        xx = pd.read_csv(export_path+site)
        xx['site'] = site[0:-4]
        xx = xx[['site', 'x_coord', 'y_coord']]
        xx=xx.head(1)
        td.append(xx)

df = pd.concat(td).dropna()
print('n_sites:', len(df))

#export site list to file
df.to_csv(site_export+'ozflux_site_locations.csv')

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.x_coord, df.y_coord), crs="EPSG:4326"
)

ax = gdf.plot(column='site', figsize=(10,10))
gdf.apply(lambda x: ax.annotate(text=x['site'],
            xy=x.geometry.centroid.coords[0],
            ha='right', fontsize=8), axis=1);

# Adding basemap might fail with max retries...something is wrong with contextily backend
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, crs='EPSG:4326', attribution='', attribution_size=1) 

## Step 4: Generate ensemble of Models

We will attempt to model a portion of the empirical uncertainty that comes from the training data. To do this, we will generate 15 models. For each iteration, two flux tower sites  will be removed from the training data and an LGBM and RF model will be fit on the remaining data.  This will result in 30 models that later we can use to make 30 predictions. The IQR envelope of our predictions will inform our uncertainity

> Note, before running this section shutdown any dask cluster that is running using `client.shutdown()`

### Analysis Parameters

In [ ]:
# client.shutdown()

In [ ]:
model_var = 'ER' #ER NEE ET GPP
n_iter = 100 #how many hyperparameter iterations to test for the final model fitting?
n_models = 15 #how many iterations of models to create (iterations of training data)?
n_cpus = 24

base = '/g/data/os22/chad_tmp/AusEFlux/'

ec_exclusions=['DalyUncleared', 'RedDirtMelonFarm', 'Loxton']

modelling_vars = ['LST_RS', 'ΔT_RS',
                  'kNDVI_RS','kNDVI_anom_RS',
                  'NDWI_RS','NDWI_anom_RS',
                  'trees_RS', 'grass_RS', 'bare_RS', 'C4_grass_RS',
                  'rain_RS', 'rain_cml3_RS', 'rain_anom_RS',
                  'rain_cml3_anom_RS', 'rain_cml6_anom_RS', 'rain_cml12_anom_RS',
                  'SRAD_RS', 'SRAD_anom_RS',
                  'Tavg_RS', 'Tavg_anom_RS',
                  'VPD_RS', 'VPD_anom_RS',
                  'VegH_RS', 'site'
                ]

### Preprocess training data

In [ ]:
import os
import pandas as pd

In [ ]:
#Comibine EC site data into a big pandas df------------------------
sites = os.listdir(f'{base}data/training_data/')
fluxes=['NEE_SOLO_EC','GPP_SOLO_EC','ER_SOLO_EC','ET_EC']
td = []
for site in sites:
    if '.csv' in site:
        if any(exc in site for exc in ec_exclusions): #don't load the excluded sites
            print('skip', site[0:-4])
            continue
        else:
            xx = pd.read_csv(f'{base}data/training_data/{site}',
                             index_col='time', parse_dates=True)
            xx['site'] = site[0:-4]
            xx = xx[fluxes+modelling_vars]
            td.append(xx)

ts = pd.concat(td).dropna() #we'll use this later

# convert pandas df into sklearn X, y --------------------------
xx = []
yy = []
for t in td:    
    t = t.dropna()  # remove NaNS
    df = t.drop(['NEE_SOLO_EC','GPP_SOLO_EC','ER_SOLO_EC'],
                axis=1) # seperate carbon fluxes
    
    df = df[modelling_vars]
    
    if model_var == 'ET':
        df_var=t[[model_var+'_EC', 'site']]
    else:
        df_var=t[[model_var+'_SOLO_EC', 'site']]
    
    x = df.reset_index(drop=True)
    y = df_var.reset_index(drop=True)
    xx.append(x)
    yy.append(y)

x = pd.concat(xx)
y = pd.concat(yy)
print(x.shape)

#export features list ----------------------------------
textfile = open(f'{base}results/variables.txt', 'w')
for element in x.columns:
    textfile.write(element + ",")
textfile.close()

### Run Step 4

Note, it will take many hours to create 30 unique models.

In [ ]:
from _ensemble_modelling import ensemble_models

In [ ]:
ensemble_models(
    base=base,
    model_var=model_var,
    x=x,
    y=y,
    n_cpus=n_cpus,
    n_iter=n_iter,
    n_models=n_models,
    verbose=True
)

### Create validaton plots

In [ ]:
from _ensemble_modelling import validation_plots

In [ ]:
validation_plots(
    base=base,
    model_var=model_var
)

### Create ensemble feature importance plots

> Note, the RF models are very slow to process, so this can take several hours to complete

In [ ]:
from _ensemble_modelling import ensemble_feature_importance

In [ ]:
ensemble_feature_importance(
    base=base,
    model_var=model_var,
    x=x,
    y=y,
    verbose=True
)

## Step 5: Predict ensemble

Using the ensemble of models, we will generate an ensemble of gridded predictions.

**Expected completion time ~45 mins/year**

### Analysis Parameters

* `model_var`: Which variable are we modelling? Must be one of 'GPP', 'ER', 'NEE', or 'ET'
* `base`: Path to where the harmonised datasets output from Step 1 are stored. 
* `results_path`: Path to store temporally stacked netcdf files i.e. where the outputs of Step 2 will be stored
* `year_start`: The first year in the series to predict. If predicting for a single year, make _year_start_ and _year_end_ the same.
* `year_end`: The last year in the series to predict. If predicting for a single year, make _year_start_ and _year_end_ the same.
* `models_folder`: where are the models stored?
* `features_list`: Where are the list of features used by the model?

In [ ]:
model_var = 'ER' #ER #NEE #ET #GPP
base = '/g/data/os22/chad_tmp/AusEFlux/'
year_start, year_end=2003, 2022
results_path = f'{base}results/predictions/ensemble/historical/{model_var}/'
models_folder = f'{base}results/models/ensemble/{model_var}/'
features_list = f'{base}results/variables.txt'

### Run Step 5


In [ ]:
from _ensemble_prediction import predict_ensemble

In [ ]:
%%time
predict_ensemble(
   base=base,
   model_var=model_var,
   models_folder=models_folder,
   features_list=features_list,
   results_path=results_path,
   year_start=year_start,
   year_end=year_end,
   compute_early=True,
   verbose=True
)

## Step 6: Combine ensembles

Ran an ensemble of predictions, now we need to compute the ensemble median and the uncertainty range.

This step will also output production ready datasets with appropriate metadata

**Expected completion time, 5 mins**

### Analysis Parameters

* `model_var`: Which variable are we combining? Must be one of 'GPP', 'ER', 'NEE', or 'ET'
* `base`: Path to where the modelling/data etc is occuring. We build the other path strings from the 'base' path to reduce the length of path strings.
* `results_path`: Path where final AusEFlux datasets will be output.
* `year_start`: The first year in the series. If running for a single year, make _year_start_ and _year_end_ the same.
* `year_end`: The last year in the series. If running for a single year, make _year_start_ and _year_end_ the same.
* `quantiles`: What quantiles are we using to determine the middle value and uncertainty range? The default is 0.05 and 0.95 for the uncertainty envelope, and 0.5 (median) for the middle estimate. You're advised not to change these.
* `predictions_folder`: where are the ensemble predictions stored? Those output from the previous step.

> There are also several metadata fields (e.g. `full_name`, `units`) that will change with the variable being modelled. Make sure you update these for each model run as these atttributes are appended to the exported netcdf files.

In [ ]:
base = '/g/data/os22/chad_tmp/AusEFlux/'
model_var = 'ER' #ER #NEE #ET #GPP
results_path = f'{base}results/AusEFlux/{model_var}/'
year_start, year_end=2003,2022
quantiles=[0.25,0.5,0.75] # interquartile range
# predictions_folder= f'{base}results/predictions/ensemble/annual_update/{year_start}/{model_var}/'
predictions_folder= f'{base}results/predictions/ensemble/historical/{model_var}/'

# metadata for netcdf attributes
full_name = 'Ecosystem Respiration'#'Gross Primary Productivity' #Net Ecosystem Exchange #Ecosystem Respiration #Evapotranspiration
version = 'v1.2'
crs='EPSG:4326'
units = 'gC/m\N{SUPERSCRIPT TWO}/month' #mm/month
description = f'AusEFlux {full_name} is created by empirically upscaling the OzFlux eddy covariance network using machine learning methods coupled with climate and remote sensing datasets. The estimates provided within this dataset were extracted from an ensemble of predictions and represent the median and uncertainty range.'


#### Create attributes dictionary

In [ ]:
attrs_dict={}
attrs_dict['nodata'] = np.nan
attrs_dict['crs'] = crs
attrs_dict['short_name'] = model_var
attrs_dict['long_name'] = full_name
attrs_dict['units'] = units
attrs_dict['version'] = version
attrs_dict['description'] = description

### Run step 6

In [ ]:
from _combine_ensemble import combine_ensemble

In [ ]:
combine_ensemble(
    base=base,
    model_var=model_var,
    results_path=results_path,
    predictions_folder=predictions_folder,
    year_start=year_start,
    year_end=year_end,
    attrs=attrs_dict,
    quantiles=quantiles,
    verbose=True
)